<a href="https://colab.research.google.com/github/airballoh87/ThinkStats2/blob/master/Run_painted_tree_bexa_boutique_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

In [ ]:
##data and stats
import pandas as pd
import seaborn as sns

##visualis
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker  # Import the ticker module

##data sources
import gspread
from google.colab import drive

##web scraping
import requests
from bs4 import BeautifulSoup

##machine learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

##NLP
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk

##saving models
import joblib

# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Get data

## Real Data

### Web Scrape from Painted Tree = Cincy only

In [ ]:
# Your login credentials
USERNAME = '105357'
PASSWORD = 'WinstonWillow1!'

# Headers for HTTP requests
http_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}

# Initialize an empty DataFrame to accumulate the data
accumulated_data = pd.DataFrame()

# Start a session to keep the cookies
with requests.Session() as session:
    # Prepare login data
    login_data = {
        'consignorId': USERNAME,
        'password': PASSWORD
    }

    # Send a POST request to the login endpoint
    login_url = 'https://cincinnati.consignoraccess.com/auth'  # Adjust this URL if necessary
    login_response = session.post(login_url, headers=http_headers, data=login_data)

    # Verify login by checking the response and cookies
    print("Login response status code:", login_response.status_code)
    print("Cookies after login:", session.cookies.get_dict())

    # Proceed to the data URL
    #data_url = 'https://cincinnati.consignoraccess.com/activity'
    data_url = 'https://cincinnati.consignoraccess.com/activity?fromDate=01/01/2023&toDate=06/30/2024&showPaidItems=true'
    response = session.get(data_url, headers=http_headers)

    # Print the response status code and content for verification
    print("Data URL response status code:", response.status_code)
    print("Data URL response content snippet:", response.content[:500])  # Print first 500 characters of the response content

    soup = BeautifulSoup(response.content, 'html.parser')
    #print("Soup object:", soup.prettify()[:500])  # Print first 500 characters of the parsed HTML

    table = soup.find('table', {'id': 'make-responsive'})

    if table:
        headers = [th.text.strip() for th in table.find('thead').find_all('th')]
        rows = []
        for row in table.find('tbody').find_all('tr'):
            columns = row.find_all('td')
            if columns:
                rows.append([col.text.strip() for col in columns])
        accumulated_data = pd.DataFrame(rows, columns=headers)

      # Add new columns 'Category', 'Type', and 'Theme' with blank values
        accumulated_data['Category'] = ''
        accumulated_data['Type'] = ''
        accumulated_data['Theme'] = ''
        accumulated_data['Location'] = 'Cincinnati'

        print("Accumulated data:", accumulated_data)
    else:
        print("Table not found in the page.")

    # Optionally save to CSV
    # accumulated_data.to_csv("sales_activity_data.csv", index=False)


### Real data - Kentucky - from google sheet


In [ ]:
gc  = gspread.authorize(creds)

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1gzZ4ffnM5Uc4cCHmAigMQ4IS3xxVokBc1wM5tQjPQcM/edit?gid=85248025#gid=85248025') ##Kentucky
sheets = gsheets.worksheet('Painted Tree Category From Website - Kentucky').get_all_values()

kentucky_sales_data = pd.DataFrame(sheets[1:], columns=sheets[0])
kentucky_sales_data['Location'] = 'Kentucky'
kentucky_sales_data

###merge data together

In [ ]:
# Concatenate accumulated_data with historical_sales_data as union
union_data = pd.concat([accumulated_data, kentucky_sales_data], axis=0, ignore_index=True)
union_data

#Clean Data

## rows and columns - general

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import joblib


# Load the dataset
new_sales_data = union_data

# Clean and convert price columns to numerical values
def clean_price(price):
    if isinstance(price, str):
        price = price.replace('(', '').replace(')', '').replace('$', '').replace(',', '')
        try:
            return float(price)
        except ValueError:
            return None
    return price

new_sales_data['Retail Price'] = new_sales_data['Retail Price'].apply(clean_price)
new_sales_data['Sold Price'] = new_sales_data['Sold Price'].apply(clean_price)
new_sales_data['Store Amount'] = new_sales_data['Store Amount'].apply(clean_price)
new_sales_data['Consignor Amount'] = new_sales_data['Consignor Amount'].apply(clean_price)

# Drop rows with missing values in the target column 'Category'
new_sales_data = new_sales_data.dropna(subset=['Category'])

#Eliminates historical rows where the was a return
new_sales_data = new_sales_data[(new_sales_data['Sold Price'] >= 0) #&
                                #(new_sales_data['Store Amount'] >= 0) &
                                #(new_sales_data['Consignor Amount'] >= 0)
                                ]


# Clean up the 'Category' column
new_sales_data['Category'] = new_sales_data['Category'].str.upper()

# Make specified updates to the 'Category' column
category_updates = {
    'MISC': 'MISCELLANEOUS',
    'SHELL': 'DECOR',
    'CLOTHES': 'CLOTHING',
    'HAT': 'HATS',
    'APPAREL': 'CLOTHING',
    'DECOR - EASTER': 'DECOR',
    'DECOR - VALENTINES': 'DECOR',
    'DOGS': 'PET',
    'HAIR ACCESSORIES': 'ACCESSORIES: HAIR',
    'HOLIDAY - EASTER': 'DECOR',
    'HOLIDAY DECOR - EASTER': 'DECOR',
    'HOME KITCHEN': 'HOME',
    'KIDS': 'KIDS ACCESSORIES',
    'KIDS APPAREL': 'KIDS CLOTHING',
    'KIDS HAIR': 'KIDS ACCESSORIES',
    'KIDS HAT': 'KIDS ACCESSORIES',
    'KIDS HATS': 'KIDS ACCESSORIES',
    'KIDS JEWELRY': 'KIDS ACCESSORIES'
}


type_updates = {

    'BUNNY': 'BUNNY DECOR',
    'COAT': 'JACKET',
    'COVER UP': 'COVERUP',
    'COWBOY HAT': 'COWBOY',
    'COWBOY HAT DECOR': 'COWBOY',
    'DOG COLLAR LARGE': 'DOG COLLAR',
    'DOG COLLAR MEDIUM': 'DOG COLLAR',
    'DOG COLLAR SMALL' : 'DOG COLLAR',
    'EAR MUFFS': 'EARMUFFS',
    'HAIR CLIP': 'HAIR CLIPS',
    'KEY CHAIN': 'KEYCHAIN',
    'ORNAMENT': 'ORNAMENTS',
    'WINTER HAT': 'WINTER',
    'WINE TUMBLER': 'TUMBLER',
    'TOTE': 'TOTE BAG',
    'SWIM': 'SWIMWEAR',
    'RING': 'RINGS',
    'BEADED NECKLACE': 'NECKLACE',
    'BOTTLE OPENER':'MISCELLANEOUS' ,
    'SPPONS': 'SPOONS',
    'SPOON HOLDER':'SPOONS'

}

theme_updates = {
    'ANIMAL PRINT': 'ANIMALS',
    'ART DECO': 'ART',
    'EMBELLISHED DENIM': 'EMBELLISHED',
    'FLOWER': 'FLORAL',
    'FLOWERS': 'FLORAL',
    'HORSE': 'HORSES',
    'MOTHERS DAY': 'MOM',
    'MUSHROOM': 'MUSHROOMS',
    'PLANTS': 'PLANT',
    'RHINESTONE': 'RHINESTONES',
    'SELF ESTEEM': 'SELF CARE',
    'ST. PATRICKS':    'ST PATRICKS DAY',
    'ST.PATRICKS DAY': 'ST PATRICKS DAY',
    'ST. PATRICKS DAY:': 'ST PATRICKS DAY',
    'SWIFTIE': 'TAYLOR SWIFT',
    'TIGER':'TIGER/BENGALS',
    'BENGALS': 'TIGER/BENGALS',
    'VALENTINE''S DAY':'VALENTINES',
    'VALENTINE': 'VALENTINES',
    'WINTER COAT':'WINTER',
    'WESTER':'WESTERN'
}

new_sales_data['Category'] = new_sales_data['Category'].replace(category_updates)
new_sales_data['Type'] = new_sales_data['Type'].replace(type_updates)
new_sales_data['Theme'] = new_sales_data['Theme'].replace(theme_updates)

##cleanup rows that aren't items
# Drop rows where 'SKU' is blank or null
new_sales_data = new_sales_data[new_sales_data['SKU'].notna() & (new_sales_data['SKU'] != '')]


new_sales_data

#Run the model


In [ ]:
#import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk
import joblib
from google.colab import drive


union_data = new_sales_data

# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Mount Google Drive
drive.mount('/content/drive')

# Preprocess text data
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'\W+', ' ', text)  # Remove punctuation
    words = text.split()
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
    return ' '.join(words)


# Filter out rows where 'Retail Price' has a value of 'Retail Price'
model_output = union_data[union_data['Retail Price'] != 'Retail Price']

model_output = model_output.drop(columns=['Type', 'Theme', 'Category'])

# Clean and convert price columns to numerical values
def clean_price(price):
    if isinstance(price, str):
        price = price.replace('(', '').replace(')', '').replace('$', '').replace(',', '')
        try:
            return float(price)
        except ValueError:
            return None
    return price

model_output['Retail Price'] = model_output['Retail Price'].apply(clean_price)
model_output['Sold Price'] = model_output['Sold Price'].apply(clean_price)
model_output['Store Amount'] = model_output['Store Amount'].apply(clean_price)
model_output['Consignor Amount'] = model_output['Consignor Amount'].apply(clean_price)

# Apply preprocessing to the 'Description' column
model_output['Description'] = model_output['Description'].apply(preprocess_text)


######category
# Load the saved TF-IDF vectorizer
tfidf_vectorizer = joblib.load('/content/drive/My Drive/Bexa/tfidf_vectorizer_item_category.pkl')
# Load the pipeline (if needed)
pipeline = joblib.load('/content/drive/My Drive/Bexa/best_model_category.pkl')
# Transform the 'Description' column using the loaded TF-IDF vectorizer
X_accumulated_tfidf = tfidf_vectorizer.transform(model_output['Description'])
# Make predictions on the new data
model_output['Predicted_Category'] = pipeline.predict(model_output['Description'])
# Calculate prediction probabilities
pred_probabilities = pipeline.predict_proba(model_output['Description'])
# Get the maximum probability for each prediction
max_probabilities = pred_probabilities.max(axis=1)
# Add the probabilities to the DataFrame
model_output['Category_prediction_probability'] = max_probabilities

######Type
# Load the saved TF-IDF vectorizer
tfidf_vectorizer = joblib.load('/content/drive/My Drive/Bexa/tfidf_vectorizer_item_type.pkl')
# Load the pipeline (if needed)
pipeline = joblib.load('/content/drive/My Drive/Bexa/best_model_type.pkl')
# Transform the 'Description' column using the loaded TF-IDF vectorizer
X_accumulated_tfidf = tfidf_vectorizer.transform(model_output['Description'])
# Make predictions on the new data
model_output['Predicted_Type'] = pipeline.predict(model_output['Description'])
# Calculate prediction probabilities
pred_probabilities = pipeline.predict_proba(model_output['Description'])
# Get the maximum probability for each prediction
max_probabilities = pred_probabilities.max(axis=1)
# Add the probabilities to the DataFrame
model_output['Type_prediction_probability'] = max_probabilities


######Theme
# Load the saved TF-IDF vectorizer
tfidf_vectorizer = joblib.load('/content/drive/My Drive/Bexa/tfidf_vectorizer_item_theme.pkl')
# Load the pipeline (if needed)
pipeline = joblib.load('/content/drive/My Drive/Bexa/best_model_theme.pkl')
# Transform the 'Description' column using the loaded TF-IDF vectorizer
X_accumulated_tfidf = tfidf_vectorizer.transform(model_output['Description'])
# Make predictions on the new data
model_output['Predicted_Theme'] = pipeline.predict(model_output['Description'])
# Calculate prediction probabilities
pred_probabilities = pipeline.predict_proba(model_output['Description'])
# Get the maximum probability for each prediction
max_probabilities = pred_probabilities.max(axis=1)
# Add the probabilities to the DataFrame
model_output['Theme_prediction_probability'] = max_probabilities



#####final cleanup

# Remove all '#' from the 'Consignor Amount' column, replace empty strings with '0', and convert to float
model_output['Consignor Amount'] = model_output['Consignor Amount'].replace('[\$,#]', '', regex=True)
model_output['Consignor Amount'] = model_output['Consignor Amount'].replace('', '0').astype(float)


# Function to ensure four-digit year
def ensure_four_digit_year(date_str):
    # Match two-digit year and convert to four-digit year
    date_str = re.sub(r'(\d{1,2}/\d{1,2}/)(\d{2})$', r'\g<1>20\2', date_str)
    return date_str

# Apply the function to the 'Date' column
model_output['Date'] = model_output['Date'].apply(ensure_four_digit_year)

# Convert the adjusted date strings to datetime objects
model_output['Date'] = pd.to_datetime(model_output['Date'], format='%m/%d/%Y', errors='coerce')

# Display the first 20 rows of the combined data
print(model_output.head(20))


#write to file on google sheets

##definition for writing to google sheets

In [ ]:
def write_dataframe_to_gsheet(dataframe, spreadsheet_name, worksheet_name):
    # Convert the DataFrame to a list of lists
    dataframe = dataframe.astype(str)
    data = [dataframe.columns.values.tolist()] + dataframe.values.tolist()

    # Try to open the existing spreadsheet
    try:
        spreadsheet = gc.open(spreadsheet_name)
    except gspread.SpreadsheetNotFound:
        spreadsheet = gc.create(spreadsheet_name)

    # Check if the worksheet already exists and delete it if it does
    try:
        worksheet = spreadsheet.worksheet(worksheet_name)
        spreadsheet.del_worksheet(worksheet)
    except gspread.WorksheetNotFound:
        pass

    # Add a new worksheet
    worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows=len(dataframe)+1, cols=len(dataframe.columns))

    # Write data to the worksheet
    worksheet.update('A1', data)

    print("Data written to Google Sheet successfully.")



In [ ]:
# Usage
spreadsheet_name = 'painted_tree_ml_training'
worksheet_name = 'Painted Tree - New model output June final - really'
write_dataframe_to_gsheet(model_output, spreadsheet_name, worksheet_name)


##Output for the training set

In [ ]:
# Create a new DataFrame with the specified columns and new blank columns
new_columns = [
    'Date', 'Description', 'Retail Price', 'Location', 'Predicted_Category',
    'Corrected_Category', 'Category_prediction_probability', 'Predicted_Type',
    'Corrected_Type', 'Type_prediction_probability', 'Predicted_Theme',
    'Corrected_Theme', 'Theme_prediction_probability'
]

# Initialize the new columns with blank values
model_output['Corrected_Category'] = ''
model_output['Corrected_Type'] = ''
model_output['Corrected_Theme'] = ''

# Select and reorder the columns
selected_data = model_output[new_columns]

from datetime import datetime

# Get the previous month and year
current_date = datetime.now()
first_day_of_current_month = datetime(current_date.year, current_date.month, 1)
previous_month_end = first_day_of_current_month - pd.DateOffset(days=1)
previous_month_start = previous_month_end.replace(day=1)
previous_month_start, previous_month_end,selected_data['Date']
# Filter by the previous month
filtered_data = selected_data[
    (selected_data['Date'] >= previous_month_start) &
    (selected_data['Date'] <= previous_month_end)
]
# Filter by probability fields
probability_filtered_data = filtered_data[
    (filtered_data['Category_prediction_probability'] < 0.67) |
    (filtered_data['Type_prediction_probability'] < 0.67) |
    (filtered_data['Theme_prediction_probability'] < 0.67)
]

## print to google sheets
spreadsheet_name = 'painted_tree_ml_training'
worksheet_name = 'Painted Tree - Predictions Review - June'
write_dataframe_to_gsheet(probability_filtered_data, spreadsheet_name, worksheet_name)

In [ ]:

# Create a new DataFrame with the specified columns and updated names
updated_columns = {
    'Date': 'Date',
    'Description': 'Description',
    'SKU': 'SKU',
    'Invoice': 'Invoice',
    'Retail Price': 'Retail Price',
    'Sold Price': 'Sold Price',
    'Store Amount': 'Store Amount',
    'Consignor Amount': 'Consignor Amount',
    'Location': 'Location',
    'Predicted_Category': 'Category',
    'Predicted_Type': 'Type',
    'Predicted_Theme': 'Theme'
}

# Select and rename the columns
updated_data = model_output.rename(columns=updated_columns)[list(updated_columns.values())]

# Display the updated DataFrame
#print(updated_data.head())
# Usage
spreadsheet_name = 'painted_tree_ml_training'
worksheet_name = 'Painted Tree - New model output June FINAL for Visualization'
write_dataframe_to_gsheet(updated_data, spreadsheet_name, worksheet_name)


#data viz